## 借用一下之前範例的Stepwise

In [350]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out=0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    initial_list = []
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded,dtype='float64')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        #backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

## import 資料

In [351]:
df = pd.read_excel(r"C:\Users\s0954\Desktop\財金計量方法\第7周-Option\選擇權價格.xlsx",sheet_name=0)
df

,日期,交割月份,履約價,買賣權,收盤價,成交量,結算價,理論價,Delta,Gamma,...,d2,N(d1),N(d2),N(-d1),N(-d2),N'(d1),台指期貨近月收盤價,台指期貨近月成交量,台銀一年定存利率,台灣加權股價指數收盤價
0,2018/01/02,201801,10500,買權,232,3573,232,232.868827,0.835868,0.001110,...,0.956819,0.835868,0.830671,0.164132,0.169329,0.247386,10709,108602,0.01035,10710.73
1,2018/01/03,201801,10600,買權,213,7398,213,225.186776,0.825081,0.001157,...,0.914282,0.825081,0.819716,0.174919,0.180284,0.257699,10789,120998,0.01035,10801.57
2,2018/01/04,201801,10600,買權,253,2615,253,261.803819,0.891525,0.000898,...,1.215581,0.891525,0.887928,0.108475,0.112072,0.186159,10837,119190,0.01035,10848.63
3,2018/01/05,201801,10700,買權,177,5641,177,190.761942,0.883612,0.001266,...,1.179024,0.883612,0.880806,0.116388,0.119194,0.195763,10857,110637,0.01035,10879.80
4,2018/01/08,201801,10700,買權,200,3123,200,219.696939,0.955646,0.000724,...,1.690400,0.955646,0.954524,0.044354,0.045476,0.093689,10886,106537,0.01035,10915.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2018/12/24,201901,9400,買權,327,250,327,311.075529,0.733579,0.000803,...,0.581250,0.733579,0.719464,0.266421,0.280536,0.328433,9620,89050,0.01035,9639.70
243,2018/12/25,201901,9300,買權,308,567,308,300.554584,0.724893,0.000824,...,0.554942,0.724893,0.710533,0.275107,0.289467,0.333736,9477,99787,0.01035,9527.09
244,2018/12/26,201901,9300,買權,291,382,291,259.181392,0.692091,0.000921,...,0.461490,0.692091,0.677776,0.307909,0.322224,0.351750,9459,116623,0.01035,9478.99
245,2018/12/27,201901,9400,買權,320,586,320,311.492238,0.735573,0.000804,...,0.587528,0.735573,0.721575,0.264427,0.278425,0.327183,9633,124097,0.01035,9641.56


$ C = f(S,\sigma,T,r)$

$ \Delta C = \color{red}{\Delta} \cdot \Delta S + \frac{\Large\color {red}\Gamma}{\Large 2} \cdot (\Delta S)^2 + \color{red}{\Large \nu} \cdot (\Delta \sigma)+\color {red}\Theta \cdot (-\Delta T) +\color{red}{\Large\rho} \cdot (\Delta r)$

vega通常用nu來代表https://brilliant.org/wiki/option-greeks-vega/

In [352]:
c = df['收盤價']
S = df['台灣加權股價指數收盤價']
sigma = df['隱含波動率(%)']
T = df['離到期日(天)']
r = df['台銀一年定存利率']
K = df['履約價']

In [353]:
X = pd.concat([c,S,sigma,T,r,K],axis=1)
X.head(5)

,收盤價,台灣加權股價指數收盤價,隱含波動率(%),離到期日(天),台銀一年定存利率,履約價
0,232,10710.73,10.097,15,0.01035,10500
1,213,10801.57,7.97728,14,0.01035,10600
2,253,10848.63,6.95896,13,0.01035,10600
3,177,10879.80,- -,12,0.01035,10700
4,200,10915.75,- -,9,0.01035,10700


In [354]:
##把非數字者換成 NaN 再做處理 https://stackoverflow.com/questions/21771133/finding-non-numeric-rows-in-dataframe-in-pandas
columns = X.columns
X = X.drop(columns,axis=1).join(df[columns].apply(pd.to_numeric,errors = 'coerce'))
X        


,收盤價,台灣加權股價指數收盤價,隱含波動率(%),離到期日(天),台銀一年定存利率,履約價
0,232,10710.73,10.097030,15,0.01035,10500
1,213,10801.57,7.977277,14,0.01035,10600
2,253,10848.63,6.958956,13,0.01035,10600
3,177,10879.80,NaN,12,0.01035,10700
4,200,10915.75,NaN,9,0.01035,10700
...,...,...,...,...,...,...
242,327,9639.70,18.866123,23,0.01035,9400
243,308,9527.09,18.255636,22,0.01035,9300
244,291,9478.99,20.692601,21,0.01035,9300
245,320,9641.56,19.179520,20,0.01035,9400


In [355]:
X = X[X[columns].notnull().all(axis=1)]
X = X.reset_index()

In [356]:
X = X.drop(columns=['index'])
X1 = X.drop(columns=['履約價'])

In [357]:
Y = X1['收盤價']
X1 = X1.drop(columns=['收盤價'])

In [358]:
form = X1.diff()[1:]
form.columns=['delta S','delta sigma','delta T','delta rho']
form['delta S^2'] = np.square(form['delta S'])
form = form[['delta S','delta S^2','delta sigma','delta T','delta rho']]
form

,delta S,delta S^2,delta sigma,delta T,delta rho
1,90.84,8251.9056,-2.119753,-1.0,0.0
2,47.06,2214.6436,-1.018321,-1.0,0.0
3,-17.54,307.6516,-1.228460,-6.0,0.0
4,-21.03,442.2609,-0.524180,-1.0,0.0
5,73.90,5461.2100,0.019359,-1.0,0.0
...,...,...,...,...,...
201,-6.46,41.7316,2.662936,-2.0,0.0
202,-112.61,12681.0121,-0.610487,-1.0,0.0
203,-48.10,2313.6100,2.436965,-1.0,0.0
204,162.57,26429.0049,-1.513081,-1.0,0.0


In [359]:
Y1 = pd.DataFrame(Y.diff()[1:])
Y1.columns=['delta C']
Y1

,delta C
1,-19.0
2,40.0
3,-163.0
4,23.0
5,-25.0
...,...
201,11.0
202,-19.0
203,-17.0
204,29.0


$ \Delta C = \color{red}{\Delta} \cdot \Delta S + \frac{\Large\color {red}\Gamma}{\Large 2} \cdot (\Delta S)^2 + \color{red}{\Large \nu} \cdot (\Delta \sigma)+\color {red}\Theta \cdot (-\Delta T) +\color{red}{\Large\rho} \cdot (\Delta r)$

In [360]:
print(sm.OLS(Y1,sm.add_constant(form)).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                delta C   R-squared:                       0.645
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     90.68
Date:                Fri, 04 Jun 2021   Prob (F-statistic):           7.75e-44
Time:                        12:22:32   Log-Likelihood:                -1021.0
No. Observations:                 205   AIC:                             2052.
Df Residuals:                     200   BIC:                             2069.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.4534      2.700      0.168      

C:\Users\s0954\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1755: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
C:\Users\s0954\Anaconda3\lib\site-packages\statsmodels\base\model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\s0954\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\s0954\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\s0954\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


不知道是不是因為隱含波動率缺值，delta S,delta S^2不顯著

In [361]:
pd.concat([Y,X1],axis=1)

,收盤價,台灣加權股價指數收盤價,隱含波動率(%),離到期日(天),台銀一年定存利率
0,232,10710.73,10.097030,15,0.01035
1,213,10801.57,7.977277,14,0.01035
2,253,10848.63,6.958956,13,0.01035
3,90,10831.09,5.730496,7,0.01035
4,113,10810.06,5.206316,6,0.01035
...,...,...,...,...,...
201,327,9639.70,18.866123,23,0.01035
202,308,9527.09,18.255636,22,0.01035
203,291,9478.99,20.692601,21,0.01035
204,320,9641.56,19.179520,20,0.01035


In [362]:
stepwise_selection(X1,Y)

Add  台銀一年定存利率                       with p-value 2.14517e-110
Add  離到期日(天)                        with p-value 3.80558e-37
Add  隱含波動率(%)                       with p-value 1.01605e-24


['台銀一年定存利率', '離到期日(天)', '隱含波動率(%)']

In [363]:
print(sm.OLS(Y,sm.add_constant(X[['台銀一年定存利率', '離到期日(天)', '隱含波動率(%)']])).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                    收盤價   R-squared:                       0.732
Model:                            OLS   Adj. R-squared:                  0.729
Method:                 Least Squares   F-statistic:                     277.3
Date:                Fri, 04 Jun 2021   Prob (F-statistic):           8.81e-59
Time:                        12:22:33   Log-Likelihood:                -1035.2
No. Observations:                 206   AIC:                             2076.
Df Residuals:                     203   BIC:                             2086.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
台銀一年定存利率    4542.9914    891.661      5.095      0.0

結果跟上面的結果類似

In [364]:
S = X['台灣加權股價指數收盤價']
K = X['履約價']
T = X['離到期日(天)']
sigma = X['隱含波動率(%)']
r = X['台銀一年定存利率']
X.head()

,收盤價,台灣加權股價指數收盤價,隱含波動率(%),離到期日(天),台銀一年定存利率,履約價
0,232,10710.73,10.097030,15,0.01035,10500
1,213,10801.57,7.977277,14,0.01035,10600
2,253,10848.63,6.958956,13,0.01035,10600
3,90,10831.09,5.730496,7,0.01035,10750
4,113,10810.06,5.206316,6,0.01035,10700


In [365]:
X = X.drop(columns=['收盤價'])
X

,台灣加權股價指數收盤價,隱含波動率(%),離到期日(天),台銀一年定存利率,履約價
0,10710.73,10.097030,15,0.01035,10500
1,10801.57,7.977277,14,0.01035,10600
2,10848.63,6.958956,13,0.01035,10600
3,10831.09,5.730496,7,0.01035,10750
4,10810.06,5.206316,6,0.01035,10700
...,...,...,...,...,...
201,9639.70,18.866123,23,0.01035,9400
202,9527.09,18.255636,22,0.01035,9300
203,9478.99,20.692601,21,0.01035,9300
204,9641.56,19.179520,20,0.01035,9400


In [366]:
def simple_regression(X,Y):
    a=[]
    for i in X:
        a.append(sm.OLS(Y,sm.add_constant(X[i])).fit())
    return a

In [367]:
results1 = simple_regression(X,Y)
for i in results1:
    a = dict(i.pvalues)
    if len(i.pvalues)==2:    
        print (list(a.keys())[1],'\n','pvalue:',i.pvalues[1],'\n','R-squared:',i.rsquared,'\n','------------')
    else:
        print (list(a.keys())[0],'\n','pvalue:',i.pvalues[0],'\n','R-squared:',i.rsquared,'\n','------------')

台灣加權股價指數收盤價 
 pvalue: 0.00029094065410785474 
 R-squared: 0.06247251375215368 
 ------------
隱含波動率(%) 
 pvalue: 9.309056208258134e-10 
 R-squared: 0.16814189255713596 
 ------------
離到期日(天) 
 pvalue: 3.8055809672080435e-37 
 R-squared: 0.5492096427237166 
 ------------
台銀一年定存利率 
 pvalue: 2.145172586745254e-110 
 R-squared: 0.0 
 ------------
履約價 
 pvalue: 1.1570491107463175e-06 
 R-squared: 0.10970294066168318 
 ------------


皆顯著

In [368]:
def add_quadratic(X,Y):
    a=[]
    for i in X:
        a.append(sm.OLS(Y,sm.add_constant(pd.concat([X[i],np.square(X[i])],axis=1))).fit())
    return a

In [369]:
results2 = add_quadratic(X,Y)

In [370]:
for i in results2:
    #print(list(i.pvalues))
    #print(list(i.pvalues.index))
    pv = list(i.pvalues)
    index = list(i.pvalues.index)
    if len(pv)==3:
        print(index[1],"\n",
              'x',"pvalue:",pv[1],"\n",
              'x^2',"pvalue:",pv[2],"\n",
              "R-squared:",i.rsquared,"\n",
              "------------"
             )
    else:
        print(index[0],"\n",
              'x',"pvalue:",pv[0],"\n",
              'x^2',"pvalue:",pv[1],"\n",
              "R-squared:",i.rsquared,"\n",
              "------------"
             )
    

台灣加權股價指數收盤價 
 x pvalue: 0.5224738218693878 
 x^2 pvalue: 0.5642038389775076 
 R-squared: 0.06401053868439877 
 ------------
隱含波動率(%) 
 x pvalue: 7.800875579077638e-07 
 x^2 pvalue: 0.0002647764415430524 
 R-squared: 0.22103823282219537 
 ------------
離到期日(天) 
 x pvalue: 2.0640528785646925e-26 
 x^2 pvalue: 1.2035594560854605e-13 
 R-squared: 0.656427058489385 
 ------------
台銀一年定存利率 
 x pvalue: 2.145172586745254e-110 
 x^2 pvalue: 2.1451725867459254e-110 
 R-squared: 1.1102230246251565e-16 
 ------------
履約價 
 x pvalue: 0.4813504758909559 
 x^2 pvalue: 0.4253968315969012 
 R-squared: 0.11249192143265885 
 ------------


波動率跟離到期日的R-square有明顯上升

## 調整變數數量 OVB

挑 $\large\sigma $,$  T$作為變數

In [371]:
model = sm.OLS(Y,sm.add_constant(pd.concat([sigma,T],axis=1))).fit()

In [372]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    收盤價   R-squared:                       0.732
Model:                            OLS   Adj. R-squared:                  0.729
Method:                 Least Squares   F-statistic:                     277.3
Date:                Fri, 04 Jun 2021   Prob (F-statistic):           8.81e-59
Time:                        12:22:34   Log-Likelihood:                -1035.2
No. Observations:                 206   AIC:                             2076.
Df Residuals:                     203   BIC:                             2086.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         47.0200      9.229      5.095      0.0

$收盤價 = 47.02+6.2788\times隱含波動率(\%) +5.7074\times離到期日(天) $

In [373]:
model2 = sm.OLS(Y,sm.add_constant(pd.concat([sigma],axis=1))).fit()

In [374]:
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                    收盤價   R-squared:                       0.168
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     41.23
Date:                Fri, 04 Jun 2021   Prob (F-statistic):           9.31e-10
Time:                        12:22:34   Log-Likelihood:                -1151.9
No. Observations:                 206   AIC:                             2308.
Df Residuals:                     204   BIC:                             2314.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        145.4389     13.896     10.467      0.0

$收盤價=145.4389+6.0192\times隱含波動率(\%)$

In [375]:
model3 = sm.OLS(T,sm.add_constant(sigma)).fit()

In [376]:
print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:                離到期日(天)   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.1131
Date:                Fri, 04 Jun 2021   Prob (F-statistic):              0.737
Time:                        12:22:35   Log-Likelihood:                -753.06
No. Observations:                 206   AIC:                             1510.
Df Residuals:                     204   BIC:                             1517.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         17.2440      2.005      8.602      0.0

$離到期日(天) = 17.2440-0.0455\times隱含波動率(\%)$

$\delta_1=-0.0455$

### $ y = \beta_0 + \beta_1 x_1+\beta_2 x_2+u $
### $ y = \beta_0 + \tilde{\beta_1} x_1+u $
### $ x_2 = \delta_0+\delta_1 x_1+v $
### $ y = \beta_0 + \beta_1 x_1+\color{red}{\beta_2 (\delta_0+\delta_1 x_1+v)}+u $

### $ \tilde{\beta_1} = \hat{\beta_1}+\hat{\beta_2}\tilde{\delta_1}$
### $ \hat{\beta_2}\tilde{\delta_1} \rightarrow bias$

$6.0192 \approx 6.2788+5.7074*(-0.0455)$

In [377]:
6.2788+5.7074*(-0.0455)

6.019113300000001